<a href="https://colab.research.google.com/github/harshil0217/NFL_Rookie_Comps/blob/main/ProfileEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import auth
from google.cloud import storage
import io
!pip install sentence-transformers
import spacy
!python -m spacy download en_core_web_sm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')
auth.authenticate_user()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 73.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
client = storage.Client(project='cmse-381')
bucket = client.get_bucket('stats_draftprospects')
blob = bucket.blob('draft_profiles.csv')
content = blob.download_as_string()
profiles = pd.read_csv(io.BytesIO(content))

In [3]:
profiles

,player_name,draft_profile,draft_year,pos_abbr,school_abbr,pick,overall,round,team_abbr,weight,height,player_image
0,Alex Smith,Nephew of Michigan State head coach John L. Sm...,2005,QB,UTAH,1.0,1.0,1.0,SF,217.0,76.0,NaN
1,Ronnie Brown,Brown has played second fiddle to Carnell Will...,2005,RB,AUB,2.0,2.0,1.0,MIA,233.0,72.0,NaN
2,Braylon Edwards,"Edwards' father, Stanley, played at Michigan a...",2005,WR,MICH,3.0,3.0,1.0,CLE,211.0,75.0,NaN
3,Cedric Benson,Benson was drafted by the Los Angeles Dodgers ...,2005,RB,TEX,4.0,4.0,1.0,CHI,222.0,71.0,NaN
4,Carnell Williams,Williams started two games and played in nine ...,2005,RB,AUB,5.0,5.0,1.0,TB,217.0,71.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4021,Jermar Jefferson,Jefferson is an efficient back who follows and...,2021,RB,ORST,30.0,257.0,7.0,DET,206.0,70.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4022,Dax Milne,Milne is a savvy route runner with average bur...,2021,WR,BYU,31.0,258.0,7.0,WSH,193.0,73.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4023,Grant Stuard,Stuard is an undersized off-the-ball linebacke...,2021,OLB,HOU,32.0,259.0,7.0,TB,225.0,72.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4024,Robert Jones,Jones started 13 games at guard at the junior ...,1992,LB,ECU,24.0,24.0,1.0,DAL,NaN,NaN,NaN


In [4]:
#remove Person, ORG, GPE, Month, Year, and Date

spacy.require_gpu()
nlp = spacy.load("en_core_web_sm")

def remove_names(text):
    doc = nlp(text)
    nouns = [ent.text for ent in doc.ents if ent.label_ in ["PERSON", "ORG", "GPE", "MONTH", "YEAR", "DATE"]]
    for noun in nouns:
        text = text.replace(noun, "")
    return text

profiles['draft_profile'] = profiles['draft_profile'].apply(remove_names)

In [5]:
profiles

,player_name,draft_profile,draft_year,pos_abbr,school_abbr,pick,overall,round,team_abbr,weight,height,player_image
0,Alex Smith,Nephew of head coach . attempted only five ...,2005,QB,UTAH,1.0,1.0,1.0,SF,217.0,76.0,NaN
1,Ronnie Brown,has played second fiddle to throughout his c...,2005,RB,AUB,2.0,2.0,1.0,MIA,233.0,72.0,NaN
2,Braylon Edwards,"' father, , played at and in the with the a...",2005,WR,MICH,3.0,3.0,1.0,CLE,211.0,75.0,NaN
3,Cedric Benson,was drafted by in the 12th round out of high...,2005,RB,TEX,4.0,4.0,1.0,CHI,222.0,71.0,NaN
4,Carnell Williams,started two games and played in nine in befo...,2005,RB,AUB,5.0,5.0,1.0,TB,217.0,71.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4021,Jermar Jefferson,is an efficient back who follows and reads bl...,2021,RB,ORST,30.0,257.0,7.0,DET,206.0,70.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4022,Dax Milne,Milne is a savvy route runner with average bur...,2021,WR,BYU,31.0,258.0,7.0,WSH,193.0,73.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4023,Grant Stuard,is an undersized off-the-ball linebacker with...,2021,OLB,HOU,32.0,259.0,7.0,TB,225.0,72.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4024,Robert Jones,Jones started 13 games at guard at the junior ...,1992,LB,ECU,24.0,24.0,1.0,DAL,NaN,NaN,NaN


In [6]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
embeddings = model.encode(profiles['draft_profile'].tolist())

In [8]:
embeddings

array([[ 0.22900032,  0.09203777, -0.23854637, ...,  0.3488453 ,
         0.00715442, -0.11541411],
       [ 0.02127212,  0.00856486, -0.08965169, ..., -0.08128558,
         0.1234724 ,  0.22944592],
       [ 0.05543428,  0.27084804,  0.05420887, ...,  0.0327423 ,
        -0.10302526, -0.00595241],
       ...,
       [ 0.14618248,  0.31020704, -0.4018049 , ..., -0.0428086 ,
         0.31793022,  0.28054267],
       [-0.03756257,  0.3042338 , -0.56531286, ...,  0.00913922,
         0.0751298 ,  0.23641735],
       [-0.11928768,  0.15213685, -0.49459118, ...,  0.3451933 ,
         0.13011096,  0.4217599 ]], dtype=float32)

In [9]:
#return positive description percentage for each draft profile
stop_words = set(stopwords.words('english'))

def preprocess(bio):
  word_tokens  = word_tokenize(bio)
  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words and w.isalpha()]
  return " ".join(filtered_sentence)

In [10]:
profiles['preprocessed_profile'] = profiles['draft_profile'].apply(preprocess)
profiles

,player_name,draft_profile,draft_year,pos_abbr,school_abbr,pick,overall,round,team_abbr,weight,height,player_image,preprocessed_profile
0,Alex Smith,Nephew of head coach . attempted only five ...,2005,QB,UTAH,1.0,1.0,1.0,SF,217.0,76.0,NaN,Nephew head coach attempted five passes freshm...
1,Ronnie Brown,has played second fiddle to throughout his c...,2005,RB,AUB,2.0,2.0,1.0,MIA,233.0,72.0,NaN,played second fiddle throughout career absence...
2,Braylon Edwards,"' father, , played at and in the with the a...",2005,WR,MICH,3.0,3.0,1.0,CLE,211.0,75.0,NaN,father played played six games freshman sophom...
3,Cedric Benson,was drafted by in the 12th round out of high...,2005,RB,TEX,4.0,4.0,1.0,CHI,222.0,71.0,NaN,drafted round high school spent two summers pl...
4,Carnell Williams,started two games and played in nine in befo...,2005,RB,AUB,5.0,5.0,1.0,TB,217.0,71.0,NaN,started two games played nine suffered broken ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4021,Jermar Jefferson,is an efficient back who follows and reads bl...,2021,RB,ORST,30.0,257.0,7.0,DET,206.0,70.0,https://a.espncdn.com/i/headshots/nfldraft/pla...,efficient back follows reads blocks well good ...
4022,Dax Milne,Milne is a savvy route runner with average bur...,2021,WR,BYU,31.0,258.0,7.0,WSH,193.0,73.0,https://a.espncdn.com/i/headshots/nfldraft/pla...,Milne savvy route runner average burst good fe...
4023,Grant Stuard,is an undersized off-the-ball linebacker with...,2021,OLB,HOU,32.0,259.0,7.0,TB,225.0,72.0,https://a.espncdn.com/i/headshots/nfldraft/pla...,undersized linebacker good speed exceptional m...
4024,Robert Jones,Jones started 13 games at guard at the junior ...,1992,LB,ECU,24.0,24.0,1.0,DAL,NaN,NaN,NaN,Jones started games guard junior college level...


In [21]:
sia = SentimentIntensityAnalyzer()

profiles['Sentiment'] = profiles['preprocessed_profile'].apply(lambda bio: sia.polarity_scores(bio))
profiles['Sentiment'] = profiles['preprocessed_profile'].apply(lambda bio: sia.polarity_scores(bio)['compound'])

In [22]:
profiles

,player_name,draft_profile,draft_year,pos_abbr,school_abbr,pick,overall,round,team_abbr,weight,height,player_image,preprocessed_profile,Sentiment
0,Alex Smith,Nephew of head coach . attempted only five ...,2005,QB,UTAH,1.0,1.0,1.0,SF,217.0,76.0,NaN,Nephew head coach attempted five passes freshm...,0.9833
1,Ronnie Brown,has played second fiddle to throughout his c...,2005,RB,AUB,2.0,2.0,1.0,MIA,233.0,72.0,NaN,played second fiddle throughout career absence...,0.4019
2,Braylon Edwards,"' father, , played at and in the with the a...",2005,WR,MICH,3.0,3.0,1.0,CLE,211.0,75.0,NaN,father played played six games freshman sophom...,0.9337
3,Cedric Benson,was drafted by in the 12th round out of high...,2005,RB,TEX,4.0,4.0,1.0,CHI,222.0,71.0,NaN,drafted round high school spent two summers pl...,0.3102
4,Carnell Williams,started two games and played in nine in befo...,2005,RB,AUB,5.0,5.0,1.0,TB,217.0,71.0,NaN,started two games played nine suffered broken ...,0.9756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4021,Jermar Jefferson,is an efficient back who follows and reads bl...,2021,RB,ORST,30.0,257.0,7.0,DET,206.0,70.0,https://a.espncdn.com/i/headshots/nfldraft/pla...,efficient back follows reads blocks well good ...,0.6597
4022,Dax Milne,Milne is a savvy route runner with average bur...,2021,WR,BYU,31.0,258.0,7.0,WSH,193.0,73.0,https://a.espncdn.com/i/headshots/nfldraft/pla...,Milne savvy route runner average burst good fe...,0.7650
4023,Grant Stuard,is an undersized off-the-ball linebacker with...,2021,OLB,HOU,32.0,259.0,7.0,TB,225.0,72.0,https://a.espncdn.com/i/headshots/nfldraft/pla...,undersized linebacker good speed exceptional m...,0.9584
4024,Robert Jones,Jones started 13 games at guard at the junior ...,1992,LB,ECU,24.0,24.0,1.0,DAL,NaN,NaN,NaN,Jones started games guard junior college level...,0.5423


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [23]:
embedding_similarity = cosine_similarity(embeddings)
sentiment_similarity = np.array(profiles['Sentiment'])

similarity_matrix = embedding_similarity * 0.8 + sentiment_similarity * 0.2






In [24]:
similarity_df = pd.DataFrame(similarity_matrix, index=profiles['player_name'], columns=profiles['player_name'])

In [25]:
similarity_df

player_name,Alex Smith,Ronnie Brown,Braylon Edwards,Cedric Benson,Carnell Williams,Adam Jones,Troy Williamson,Antrel Rolle,Carlos Rogers,Mike Williams,...,Chris Garrett,Marquiss Spencer,Pressley Harvin III,Kawaan Baker,Kylin Hill,Jermar Jefferson,Dax Milne,Grant Stuard,Robert Jones,Tim Jones
player_name,,,,,,,,,,,,,,,,,,,,,
Alex Smith,0.996660,0.549509,0.695533,0.567921,0.676308,0.503225,0.776247,0.695947,0.564324,0.711717,...,0.436704,0.481335,0.239842,0.392304,0.541315,0.573483,0.433616,0.542245,0.479384,0.463498
Ronnie Brown,0.665789,0.880380,0.693703,0.551757,0.791071,0.498780,0.670131,0.707429,0.601586,0.567845,...,0.422794,0.524979,0.247903,0.512191,0.586387,0.605663,0.512086,0.642715,0.510426,0.557277
Braylon Edwards,0.705453,0.587343,0.986740,0.618241,0.664761,0.502339,0.786536,0.747496,0.651878,0.667179,...,0.341559,0.490083,0.229311,0.509452,0.538797,0.529932,0.411444,0.547744,0.460469,0.458186
Cedric Benson,0.702541,0.570097,0.742941,0.862040,0.674289,0.452688,0.732442,0.751012,0.599115,0.635037,...,0.350841,0.526002,0.223715,0.477187,0.587011,0.600643,0.505438,0.644111,0.488495,0.533943
Carnell Williams,0.677848,0.676331,0.656381,0.541209,0.995120,0.551359,0.694909,0.678590,0.562468,0.568510,...,0.366582,0.410700,0.227882,0.434166,0.512328,0.549100,0.486293,0.529058,0.458258,0.472469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jermar Jefferson,0.638203,0.554103,0.584732,0.530743,0.612280,0.398393,0.569355,0.736929,0.536758,0.541608,...,0.499848,0.690974,0.217577,0.605357,0.726582,0.931940,0.622993,0.772779,0.571738,0.664615
Dax Milne,0.477276,0.439466,0.445184,0.414478,0.528413,0.321965,0.474887,0.562571,0.473696,0.375061,...,0.357919,0.610890,0.198150,0.518279,0.553638,0.601933,0.953000,0.692054,0.495672,0.618176
Grant Stuard,0.547225,0.531415,0.542804,0.514471,0.532498,0.404104,0.542902,0.749054,0.561984,0.505217,...,0.491132,0.695032,0.238719,0.645287,0.667932,0.713039,0.653374,0.991680,0.612514,0.708653


In [26]:
similarity_df.sort_values(by = 'Joe Burrow', ascending = False).head(10)


player_name,Alex Smith,Ronnie Brown,Braylon Edwards,Cedric Benson,Carnell Williams,Adam Jones,Troy Williamson,Antrel Rolle,Carlos Rogers,Mike Williams,...,Chris Garrett,Marquiss Spencer,Pressley Harvin III,Kawaan Baker,Kylin Hill,Jermar Jefferson,Dax Milne,Grant Stuard,Robert Jones,Tim Jones
player_name,,,,,,,,,,,,,,,,,,,,,
Joe Burrow,0.580177,0.480405,0.571062,0.487828,0.535045,0.404775,0.613575,0.720498,0.489631,0.617457,...,0.487759,0.597705,0.248288,0.642694,0.592789,0.642842,0.516123,0.766177,0.512128,0.601039
Nate Stanley,0.641721,0.532806,0.670790,0.567694,0.568968,0.440796,0.675149,0.778093,0.566737,0.658384,...,0.484082,0.635013,0.272877,0.636489,0.676333,0.683679,0.521297,0.795283,0.559831,0.622985
Xavier McKinney,0.637392,0.558861,0.657899,0.547766,0.570592,0.443006,0.667298,0.789907,0.559152,0.602937,...,0.506912,0.678207,0.260067,0.682532,0.680440,0.695556,0.596301,0.825985,0.546105,0.655010
Marlon Davidson,0.633866,0.540873,0.588893,0.519619,0.572059,0.489296,0.663846,0.744816,0.529833,0.604015,...,0.553026,0.619187,0.264357,0.582806,0.623947,0.663580,0.558967,0.808253,0.578630,0.605469
Zach Wilson,0.644499,0.521669,0.625348,0.542772,0.563367,0.403499,0.671119,0.729506,0.534284,0.622334,...,0.479993,0.633329,0.259832,0.625919,0.662731,0.675882,0.564377,0.765614,0.551737,0.633625
Trevor Lawrence,0.608583,0.481708,0.600471,0.520443,0.500553,0.411360,0.645068,0.702021,0.456459,0.552645,...,0.460421,0.551624,0.262686,0.606883,0.614463,0.628647,0.502002,0.749310,0.492048,0.621233
Zaven Collins,0.581407,0.533924,0.609116,0.552292,0.565386,0.461629,0.627251,0.754058,0.544564,0.548840,...,0.474739,0.670640,0.267500,0.667232,0.649618,0.670476,0.638624,0.891373,0.560091,0.659431
Isaiah Simmons,0.651981,0.514262,0.643019,0.555094,0.568590,0.425654,0.661859,0.800642,0.531099,0.669373,...,0.497226,0.676286,0.261759,0.661719,0.691517,0.699136,0.601298,0.837897,0.605063,0.663886
Jamar Johnson,0.609119,0.577166,0.615545,0.557244,0.583900,0.463236,0.630552,0.765161,0.576510,0.608855,...,0.514956,0.676493,0.232657,0.641915,0.679022,0.742696,0.606231,0.858930,0.611937,0.680748


In [27]:
similarity_df

player_name,Alex Smith,Ronnie Brown,Braylon Edwards,Cedric Benson,Carnell Williams,Adam Jones,Troy Williamson,Antrel Rolle,Carlos Rogers,Mike Williams,...,Chris Garrett,Marquiss Spencer,Pressley Harvin III,Kawaan Baker,Kylin Hill,Jermar Jefferson,Dax Milne,Grant Stuard,Robert Jones,Tim Jones
player_name,,,,,,,,,,,,,,,,,,,,,
Alex Smith,0.996660,0.549509,0.695533,0.567921,0.676308,0.503225,0.776247,0.695947,0.564324,0.711717,...,0.436704,0.481335,0.239842,0.392304,0.541315,0.573483,0.433616,0.542245,0.479384,0.463498
Ronnie Brown,0.665789,0.880380,0.693703,0.551757,0.791071,0.498780,0.670131,0.707429,0.601586,0.567845,...,0.422794,0.524979,0.247903,0.512191,0.586387,0.605663,0.512086,0.642715,0.510426,0.557277
Braylon Edwards,0.705453,0.587343,0.986740,0.618241,0.664761,0.502339,0.786536,0.747496,0.651878,0.667179,...,0.341559,0.490083,0.229311,0.509452,0.538797,0.529932,0.411444,0.547744,0.460469,0.458186
Cedric Benson,0.702541,0.570097,0.742941,0.862040,0.674289,0.452688,0.732442,0.751012,0.599115,0.635037,...,0.350841,0.526002,0.223715,0.477187,0.587011,0.600643,0.505438,0.644111,0.488495,0.533943
Carnell Williams,0.677848,0.676331,0.656381,0.541209,0.995120,0.551359,0.694909,0.678590,0.562468,0.568510,...,0.366582,0.410700,0.227882,0.434166,0.512328,0.549100,0.486293,0.529058,0.458258,0.472469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jermar Jefferson,0.638203,0.554103,0.584732,0.530743,0.612280,0.398393,0.569355,0.736929,0.536758,0.541608,...,0.499848,0.690974,0.217577,0.605357,0.726582,0.931940,0.622993,0.772779,0.571738,0.664615
Dax Milne,0.477276,0.439466,0.445184,0.414478,0.528413,0.321965,0.474887,0.562571,0.473696,0.375061,...,0.357919,0.610890,0.198150,0.518279,0.553638,0.601933,0.953000,0.692054,0.495672,0.618176
Grant Stuard,0.547225,0.531415,0.542804,0.514471,0.532498,0.404104,0.542902,0.749054,0.561984,0.505217,...,0.491132,0.695032,0.238719,0.645287,0.667932,0.713039,0.653374,0.991680,0.612514,0.708653


In [28]:
joe_shiesty = similarity_df.loc['Joe Burrow']

In [29]:
joe_shiesty.sort_values(ascending = False).head(10)

,Joe Burrow
player_name,
Joe Burrow,0.982340
Nate Stanley,0.886137
Xavier McKinney,0.868375
Marlon Davidson,0.866631
Zach Wilson,0.866164
Trevor Lawrence,0.859100
Zaven Collins,0.852643
Isaiah Simmons,0.831882
Blake Bortles,0.831668


In [ ]:
# generate the three most similar players to Joe Burrow, making sure they match his position and were drafted in the same round

